## 0. Simulate some data and fit an emulator

In [1]:
import torch

from autoemulate.experimental.simulations.projectile import ProjectileMultioutput
from autoemulate.experimental.emulators.gaussian_process.exact import (
    GaussianProcessExact,
)

In [2]:
sim = ProjectileMultioutput()
x = sim.sample_inputs(100)
y = sim.forward_batch(x)

Running simulations: 100%|██████████| 100/100 [00:00<00:00, 1298.58it/s]

Successfully completed 100/100 simulations (100.0%)


In [3]:
gp = GaussianProcessExact(x, y)
gp.fit(x, y)

## 1. Simple HMC example.

In [4]:
from autoemulate.experimental.calibration.hmc import HMCCalibrator

Lets start with an observation inside the training range, we should be able to recover the input parameters.

In other words, just pick as observation one row of our simulated values.

In [5]:
idx = 4
observations = {
    param: val for param, val in 
    zip(sim.output_names, y[idx, :])
}

In [6]:
# use the simulator parameter_range 
hmc = HMCCalibrator(gp, sim.parameters_range, observations, 1.0)

Run MCMC (note that below we have set the number of MCMC steps to a very low number, don't expect convergence).

In [7]:
mcmc = hmc.run_mcmc(warmup_steps=10, num_samples=100)

Sample: 100%|██████████| 110/110 [00:51,  2.12it/s, step size=9.25e-05, acc. prob=0.485]


The returned Pyro MCMC object has methods for accessing the generated samples (`mcmc.get_samples()`) or, as shown below, to get just their summary statistics.

In [8]:
mcmc.summary()


                mean       std    median      5.0%     95.0%     n_eff     r_hat
         c     -0.45      0.29     -0.46     -0.82      0.04      2.49      2.70
        v0    320.89      4.87    319.56    315.54    324.75     27.76      1.01

Number of divergences: 91


Compare to the input that generated  our "observed" value...

In [9]:
x[idx]

tensor([ -3.1849, 249.8333])

## 2. Combining this with sensitivity analysis and history matching.

The `HMCCalibrator` object has an option to provide a list of parameters to calibrate. These can be the result of `SensitivityAnalysis`, or just a list provided by the user.

Similarly, the user provides parameter ranges from which to sample or set the parameter values. This can be simply the range of the simulator or one can use `HistoryMatching` to reduce the parameter range and pass that to the `HMCCalibrator` instead. 

Below we demonstrate how to do both.

In [10]:
from autoemulate.experimental.sensitivity_analysis import SensitivityAnalysis
from autoemulate.experimental.calibration.history_matching import HistoryMatching

1. Run sensitivity analysis and get top N parameters (here we just get the top 1).

In [11]:
problem = {
        "num_vars": 2,
        "names": sim.param_names,
        "bounds": sim.param_bounds,
    }
sa = SensitivityAnalysis(gp, problem=problem)
df = sa.run("sobol")

/Users/rjersakova/Documents/Projects/autoemulate/.venv/lib/python3.12/site-packages/SALib/util/__init__.py:274: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  names = list(pd.unique(groups))
/Users/rjersakova/Documents/Projects/autoemulate/.venv/lib/python3.12/site-packages/SALib/util/__init__.py:274: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  names = list(pd.unique(groups))


Notice that the output is just a list of strings, the user does not need to run SA to set a subset of parameters to calibrate.

In [12]:
top_param = sa.top_n_params(df, 1)
top_param

['v0']

2. Run history matching and generate new parameter bounds from NROY samples.

In [39]:
# start with some GP predictions
x_new = sim.sample_inputs(20)
output = gp.predict(torch.tensor(x_new, dtype=torch.float32))
pred_means, pred_vars = (
    output.mean.float().detach(),
    output.variance.float().detach(),
)

/var/folders/bj/kdwy1bhj3h728lr5xdj19yd40000gr/T/ipykernel_70638/3687513851.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  output = gp.predict(torch.tensor(x_new, dtype=torch.float32))


In [43]:
# generate NROY samples
hm = HistoryMatching(
    # add noise to observations
    observations={k: [v, 10.0] for k,v in observations.items()},
    threshold=5.0,
    rank=2
)
implausability = hm.calculate_implausibility(pred_means, pred_vars)
nroy_samples = hm.get_nroy(implausability, x_new)
nroy_samples

tensor([[ -2.4475, 983.5627],
        [ -4.9263, 600.8964]])

The newly generated range is slightly narrower than the range of the simulator.

In [44]:
# get new param bounds
nroy_param_range = hm.generate_param_bounds(nroy_samples, param_names = sim.param_names)
nroy_param_range

{'c': [tensor(-5.0503), tensor(-2.3235)],
 'v0': [tensor(581.7631), tensor(1002.6960)]}

3. Pass results to the HMCCalibrator object.

In [45]:
hmc = HMCCalibrator(
    gp, 
    nroy_param_range, 
    observations, 
    1.0,
    top_param
    )

In [46]:
mcmc = hmc.run_mcmc(warmup_steps=10, num_samples=100)

Sample: 100%|██████████| 110/110 [00:00, 406.11it/s, step size=1.21e+03, acc. prob=0.000]


In [47]:
mcmc.summary()


                mean       std    median      5.0%     95.0%     n_eff     r_hat
        v0    581.76      0.00    581.76    581.76    581.76      0.50       nan

Number of divergences: 100
